In [20]:
from tqdm import tqdm
from datetime import date
import bs4, requests, json

In [2]:
irr = r'https://burma.irrawaddy.com/category/news'
# get original page
soup = bs4.BeautifulSoup(requests.get(irr).content, 'lxml')
# get trending links
trending = soup.select('.menu-item.trending-tag')
trendlinks = [t.a['href'] for t in trending]
print('There are {} links.'.format(len(trendlinks)))

There are 7 links.


In [12]:
art_url_list = []
article_list = []
for link in tqdm(trendlinks):
    current_page = bs4.BeautifulSoup(requests.get(link).content, 'lxml')
    articles = current_page.select('.article-header')
    art_url_list.extend([article_header.a['href'] for article_header in articles])

100%|██████████| 7/7 [00:12<00:00,  1.81s/it]


In [13]:
print(len(art_url_list), art_url_list[-1])

68 https://burma.irrawaddy.com/lifestyle/health/2020/01/16/213095.html


In [14]:
# Take Unique
art_url_list_unq = list(set(art_url_list))
print(len(art_url_list), len(art_url_list_unq))

68 60


In [24]:
N = 50 # we are only taking 50

article_list = []
for art_url in tqdm(art_url_list_unq[:N]):
    article = bs4.BeautifulSoup(requests.get(art_url).content, 'lxml')
    art_title = article.select('.article-title')[0].h1.text
    art_body = article.select('.article-entry')
    article_list.append({
        'url': art_url,
        'title': art_title,
        'body': '\n'.join(para.text for para in art_body[0].find_all('p'))
    })

100%|██████████| 50/50 [01:37<00:00,  1.96s/it]


In [25]:
with open('data/irr_{}.json'.format(date.today()), 'w', encoding='utf-8') as f:
    json.dump(article_list, f, ensure_ascii=False, indent=4)